# Collate Trades from ProfitView Bot

This notebook fetches executed-trade and position data from a running ProfitView bot HTTP endpoints and produces simple summaries and a small equity curve plot.

Configuration: set `BASE_ENDPOINT` to the bot base URL including the webhook secret. Example: `http://localhost:8000/trading/bot/REPLACE_WITH_SECRET`.

Make sure the bot is running and that the endpoints `/status` and `/positions` (or similar) are available under the base endpoint. If your bot exposes different route names, update the helper call paths below.

In [ ]:
# Example single-request URL (for quick testing) - not executed
# Example: f"{BASE_ENDPOINT.rstrip('/')}/positions?venue=WooPaper"

In [1]:
# Configuration - set these before running the remaining cells
BASE_ENDPOINT = "https://profitview.net/trading/signal/1T0zAuGV4yoUgA/"  # update to your bot base URL (include webhook secret if required)
# Optional: override specific endpoint paths if your bot uses different names
STATUS_PATH = 'status'
POSITIONS_PATH = 'positions'
ORDERBLOCKS_PATH = 'orderblocks'  # optional endpoint exposing order blocks or recent OBs
# Optional headers (if your deployment requires token or API key in headers)
HEADERS = {'Accept': 'application/json'}

In [2]:
# Helpers: imports and tiny request helpers
import requests
import pandas as pd
import matplotlib.pyplot as plt
from typing import Any, Dict, Optional

def fetch_endpoint(path: str, params: Optional[dict] = None) -> Dict[str, Any]:
    url = f"{BASE_ENDPOINT.rstrip('/')}/{path.lstrip('/')}"
    try:
        r = requests.get(url, headers=globals().get('HEADERS', {}), params=params, timeout=10, verify=True)
        r.raise_for_status()
        # attempt to parse JSON, otherwise return text
        try:
            return r.json()
        except Exception:
            return {'_text': r.text}
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return {}

In [5]:
# Fetch data from the bot
status_raw = fetch_endpoint(STATUS_PATH)
positions_raw = fetch_endpoint(POSITIONS_PATH)

# Many ProfitView endpoints return an envelope like {'status': ..., 'data': ...} --
# unwrap 'data' if present so downstream cells work whether the response is enveloped or raw.
def _unwrap(resp):
    if isinstance(resp, dict) and 'data' in resp and resp.get('data') is not None:
        return resp['data']
    return resp

status = _unwrap(status_raw)
positions = _unwrap(positions_raw)

# Print diagnostics so you can see the actual nested structure and adapt parsing.
print('status keys (raw):', list(status_raw.keys()) if isinstance(status_raw, dict) else type(status_raw))
print('positions keys (raw):', list(positions_raw.keys()) if isinstance(positions_raw, dict) else type(positions_raw))
print('status type after unwrap:', type(status))
print('positions type after unwrap:', type(positions))
if isinstance(positions, dict):
    print('positions keys after unwrap:', list(positions.keys()))
else:
    print('positions is non-dict (e.g., list) length:', len(positions) if hasattr(positions, '__len__') else 'unknown')

# Print a truncated JSON sample of the positions data to help decide mapping (first 1000 chars).
import json
try:
    sample = positions
    s = json.dumps(sample, default=str) if sample is not None else ''
    print('positions sample (truncated):')
    print(s[:1000])
except Exception as e:
    print('could not serialise sample:', e)

status keys (raw): ['status', 'data']
positions keys (raw): ['status', 'data']
status type after unwrap: <class 'dict'>
positions type after unwrap: <class 'dict'>
positions keys after unwrap: ['open_positions', 'closed_positions', 'all_positions_summary', 'summary', 'parameters']
positions sample (truncated):
{"open_positions": [], "closed_positions": [], "all_positions_summary": [], "summary": {"total_positions": 0, "open_count": 0, "closed_count": 0, "pending_orders": 0}, "parameters": {"symbol": "PERP_ETH_USDT", "venue": "WooLive", "candle_level": "15m", "risk_per_trade_percent": 1.5, "use_fixed_capital": true, "max_position_size_usd": 100000.0, "stop_loss_multiplier": 1.2, "take_profit_multiplier": 15.0, "max_concurrent_positions": 3, "trailing_stop_activation": 2.0, "trailing_stop_percent": 1.0, "trailing_stop_buffer_candles": 10, "entry_price_mode": "Close", "ema_period": 12, "entry_diff_long_pct": 0.02, "entry_diff_short_pct": 0.03, "commission_percent": 0.1, "slippage_percent"

In [3]:
# Helper to POST updated parameters to the running bot
def update_params(params: dict) -> dict:
    """POST a JSON body to the bot `/update_params` endpoint and return parsed JSON or text wrapper."""
    url = f"{BASE_ENDPOINT.rstrip('/')}/update_params"
    try:
        r = requests.post(url, headers=globals().get('HEADERS', {}), json=params, timeout=10)
        r.raise_for_status()
        try:
            return r.json()
        except Exception:
            return {'_text': r.text}
    except Exception as e:
        print(f"Error posting to {url}: {e}")
        return {}

# Example usage:
resp = update_params({'take_profit_multiplier': 8.0})
print(resp)

{'status': 'success', 'data': {'status': 'success', 'updated': ['take_profit_multiplier: 15.0 → 8.0'], 'errors': [], 'message': 'Updated 1 parameter(s)'}}


In [ ]:
# Normalize positions data into DataFrames
def ensure_list(x):
    if x is None:
        return []
    if isinstance(x, list):
        return x
    return [x]

# The variable may be a dict with 'open'/'closed', or it may already be a list of positions,
# or a dict with lists nested under other keys. Try common patterns with best-effort fallbacks.
open_positions_raw = None
closed_positions_raw = None

if isinstance(positions, dict):
    # common key names
    open_positions_raw = positions.get('open') or positions.get('open_positions') or positions.get('active')
    closed_positions_raw = positions.get('closed') or positions.get('closed_positions') or positions.get('history')
    # If the data dict itself contains a flat list under an unexpected key, collect any list-valued entries as a fallback
    if not open_positions_raw and not closed_positions_raw:
        maybe = []
        for k,v in positions.items():
            if isinstance(v, list):
                maybe.extend(v)
        if maybe:
            closed_positions_raw = maybe

elif isinstance(positions, list):
    # treat a top-level list as closed/historical trades
    closed_positions_raw = positions

# Normalize to dataframes
open_positions = pd.json_normalize(ensure_list(open_positions_raw)) if open_positions_raw else pd.DataFrame()
closed_positions = pd.json_normalize(ensure_list(closed_positions_raw)) if closed_positions_raw else pd.DataFrame()

# Try to find fills under a few common keys
fills = pd.DataFrame()
if isinstance(positions, dict):
    if 'fills' in positions:
        fills = pd.json_normalize(ensure_list(positions.get('fills')))
    elif 'trades' in positions:
        fills = pd.json_normalize(ensure_list(positions.get('trades')))

print('open positions rows:', len(open_positions))
print('closed positions rows:', len(closed_positions))
print('fills rows:', len(fills))

open positions: 0
closed positions: 0
fills rows: 0


In [ ]:
# Produce simple summaries and a small equity curve plot
# Try to extract a per-trade pnl column from common keys
def extract_pnl(df: pd.DataFrame) -> pd.Series:
    for col in ['pnl', 'pnl_dollars', 'realized_pnl', 'profit']:
        if col in df.columns:
            return pd.to_numeric(df[col], errors='coerce').fillna(0)
    return pd.Series([], dtype=float)

closed_pnl = extract_pnl(closed_positions)
total_trades = len(closed_positions)
total_pnl = closed_pnl.sum() if not closed_pnl.empty else 0.0
avg_pnl = closed_pnl.mean() if not closed_pnl.empty else 0.0

# Fees (best-effort)
fee_cols = [c for c in closed_positions.columns if 'fee' in c.lower() or 'commission' in c.lower()]
if fee_cols:
    total_fees = closed_positions[fee_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1).sum()
else:
    total_fees = None

print(f'Total closed trades: {total_trades}')
print(f'Total PnL: {total_pnl}')
print(f'Average PnL per closed trade: {avg_pnl}')
print(f'Total fees (if found): {total_fees}')

# Equity curve (cumulative closed trade PnL)
if not closed_pnl.empty:
    equity = closed_pnl.cumsum()
    plt.figure(figsize=(8,4))
    plt.plot(equity.index, equity.values, marker='o')
    plt.title('Equity curve (closed trades cumulative PnL)')
    plt.xlabel('Trade index')
    plt.ylabel('Cumulative PnL')
    plt.grid(True)
    plt.show()
else:
    print('No closed trade PnL data available to plot.')
    # Helpful hint when the positions payload is empty:
    print("The positions payload contained no closed trades (closed=[]). If you expect historical trades, check that your bot exposes them under 'closed' or 'history' keys.")
    print("If your API uses different field names, here's a small mapping template you can adapt:")

# Optionally write an empty CSV scaffold to edit mapping in a spreadsheet editor
try:
    if closed_positions is None or len(closed_positions)==0:
        # write an empty scaffold with expected columns for manual mapping
        scaffold = pd.DataFrame(columns=['symbol','entry_time','exit_time','pnl_dollars','fee'])
        scaffold.to_csv('closed_positions_schema.csv', index=False)
        print('Wrote scaffold file closed_positions_schema.csv')
    else:
        # Ensure common columns exist with safe conversions
        closed_positions['symbol'] = closed_positions.get('market') or closed_positions.get('symbol')
        closed_positions['entry_time'] = closed_positions.get('entry_time') or closed_positions.get('opened_at')
        closed_positions['exit_time'] = closed_positions.get('exit_time') or closed_positions.get('closed_at')
        closed_positions['pnl_dollars'] = pd.to_numeric(closed_positions.get('pnl_dollars') or closed_positions.get('profit') or closed_positions.get('realized_pnl'), errors='coerce')
        closed_positions['fee'] = pd.to_numeric(closed_positions.get('fee') or closed_positions.get('commission'), errors='coerce')
        print('Prepared closed_positions dataframe with common columns (symbol, entry_time, exit_time, pnl_dollars, fee)')
except Exception as e:
    print('Could not write scaffold CSV:', e)

Total closed trades: 0
Total PnL: 0.0
Average PnL per closed trade: 0.0
Total fees (if found): None
No closed trade PnL data available to plot.


## How to run
1. Update the `BASE_ENDPOINT` value in the configuration cell to point to your running bot (include webhook secret if required).
2. Run all cells (Kernel -> Restart & Run All) or run them top-to-bottom.
3. If endpoints are unavailable or named differently, update `STATUS_PATH`/`POSITIONS_PATH` or inspect the `status`/`positions` raw outputs (printed in the fetch cell) to adapt parsing.

Notes: This notebook uses best-effort heuristics to locate PnL and fee columns. Adjust the extraction logic to match the exact JSON structure your bot returns for more accurate reporting.